# Data Quality con Great Expectations

In [1]:
#! pip install great_expectations

In [2]:
import os
import sys
import pandas as pd
import datetime
import re
from ruamel import yaml
from ruamel.yaml import YAML
sys.path.append('../../')

## Ge
import great_expectations as gx
import great_expectations as gx
import great_expectations.jupyter_ux
from great_expectations.cli.datasource import sanitize_yaml_and_save_datasource, check_if_datasource_name_exists
from great_expectations.core.batch import RuntimeBatchRequest
from great_expectations.core.expectation_configuration import ExpectationConfiguration
from great_expectations.data_context.types.resource_identifiers import ExpectationSuiteIdentifier
from great_expectations.exceptions import DataContextError
from great_expectations.checkpoint.types.checkpoint_result import CheckpointResult


from pprint import pprint




2023-02-10T15:08:36+0000 - INFO - Great Expectations logging enabled at 20 level by JupyterUX module.


## Data Quality 

Dopo aver completato la fase di Data Ingestion tramite lo scraping il secondo step è quello di valutare la qualità dei dati raccolti.  
Il dataframe studiato presenta 18 colonne. Ogni colonna presenta delle differenze dalle altre e ci si aspetta determinati valori da queste.

Dopo aver settato:
1. la datasource
2. suite (e le relative expectations)
3. il checkpoint

Si condurrà una validazione del sample

In [3]:
for e in os.listdir('../data'):
    print(e)

scraping_all_amsterdam_company_template.csv
scraping_all_amsterdam_job_template.csv
scraping_all_amsterdam_template.csv
scraping_all_londra_080223.csv
scraping_all_londra_company_080223.csv
scraping_all_londra_job_080223.csv


In [4]:
df = pd.read_csv('../data/scraping_all_londra_080223.csv')


In [5]:
df = df.rename(columns={'Sede centrale':'sede_centrale',
                                        'Dimensioni':'dimensioni',
                                        'Fondata nel' : 'fondata_nel',
                                        'Tipo':'tipo',
                                        'Entrate':'entrate',
                                        # job_link
                                        'Opportunità di carriera': 'oppurtunita_carriera',
                                        'Stipendio e benefit':'stipendio_e_benefit',
                                        'Cultura e valori':'cultura_e_valori',
                                        'Dirigenti senior':'dirigenti_senior',
                                        'Equilibrio lavoro/vita privata':'equilibrio_lavoro_vita_privata',
                                        'Settore':'settore',
                                        'Segmento':'segmento'})
df

,Unnamed: 0,company,job_title,location,company_rating,job_age,job_link,oppurtunita_carriera,stipendio_e_benefit,cultura_e_valori,dirigenti_senior,equilibrio_lavoro_vita_privata,sede_centrale,dimensioni,fondata_nel,tipo,settore,segmento,entrate
0,0,Secret Intelligence Service,Intelligence Officer,"Londra, Inghilterra","<span class=""css-2lqh28 e1cjmv6j1"">3.8<i class=""css-0 e1cjmv6j0""></i></span>",2 g,https://www.glassdoor.it/partner/jobListing.htm?pos=101&ao=1136043&s=58&guid=0000018631bc9a96b5de81e85a66ef30&src=GD_JOB_AD&t=SR&vt=w&cs=1_4c78bf95&cb=1675871690366&jobListingId=1008454495320&jrtk=3-0-1goorp6m52f3a001-1goorp6mritnc800-4400af47f1718840-,3.5,3.3,3.2,3.2,3.2,"Londra, Regno Unito",Sconosciuto,1909.0,Sconosciuto,Enti governativi,Istituzioni governative e pubblica amministrazione,Sconosciuto/Non applicabile
1,1,Vue,Assistant Customer Relations,"Londra, Inghilterra","<span class=""css-2lqh28 e1cjmv6j1"">3.3<i class=""css-0 e1cjmv6j0""></i></span>",9 g,https://www.glassdoor.it/partner/jobListing.htm?pos=102&ao=1136043&s=58&guid=0000018631bc9a96b5de81e85a66ef30&src=GD_JOB_AD&t=SR&vt=w&ea=1&cs=1_b4d44f6b&cb=1675871690366&jobListingId=1008437630428&jrtk=3-0-1goorp6m52f3a001-1goorp6mritnc800-3518e8f62f399b6c-,2.6,3.2,3.1,2.9,3.2,"Londra, Regno Unito",Da 1.001 a 5.000 dipendenti,1995.0,Azienda - Settore privato,Cultura e spettacolo,"Arte, intrattenimento e attività ricreative",$ 100 - $ 500 milioni (USD)
2,2,Four Seasons,Receptionist,"Londra, Inghilterra","<span class=""css-2lqh28 e1cjmv6j1"">4<i class=""css-0 e1cjmv6j0""></i></span>",15 g,https://www.glassdoor.it/partner/jobListing.htm?pos=103&ao=1136043&s=58&guid=0000018631bc9a96b5de81e85a66ef30&src=GD_JOB_AD&t=SR&vt=w&cs=1_d8cda81f&cb=1675871690366&jobListingId=1008425685308&jrtk=3-0-1goorp6m52f3a001-1goorp6mritnc800-6c303771349254bd-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,DogB&B,Dog Walker And Sitter,"Londra, Inghilterra",NaN,Oltre 30 g,https://www.glassdoor.it/partner/jobListing.htm?pos=104&ao=1136043&s=58&guid=0000018631bc9a96b5de81e85a66ef30&src=GD_JOB_AD&t=SR&vt=w&ea=1&cs=1_5056cc47&cb=1675871690366&jobListingId=1008274188534&jrtk=3-0-1goorp6m52f3a001-1goorp6mritnc800-60db12155bab82dc-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Lidl,Assistant Customer Relations,"Londra, Inghilterra","<span class=""css-2lqh28 e1cjmv6j1"">3.5<i class=""css-0 e1cjmv6j0""></i></span>",5 g,https://www.glassdoor.it/partner/jobListing.htm?pos=105&ao=1136043&s=58&guid=0000018631bc9a96b5de81e85a66ef30&src=GD_JOB_AD&t=SR&vt=w&cs=1_5bd99ae5&cb=1675871690366&jobListingId=1008447842302&jrtk=3-0-1goorp6m52f3a001-1goorp6mritnc800-bc55c0b220a49cff-,3.3,3.6,3.1,3.0,2.8,"Karlsruhe, Germania",Oltre 10.000 dipendenti,1973.0,Azienda - Settore privato,Negozi di alimentari,Vendita al dettaglio e all''ingrosso,Più di $ 10 miliardi (USD)
5,5,WHSmith,Assistant Sales Representative,"Londra, Inghilterra","<span class=""css-2lqh28 e1cjmv6j1"">3.1<i class=""css-0 e1cjmv6j0""></i></span>",25 g,https://www.glassdoor.it/partner/jobListing.htm?pos=106&ao=1136043&s=58&guid=0000018631bc9a96b5de81e85a66ef30&src=GD_JOB_AD&t=SR&vt=w&cs=1_83cbf210&cb=1675871690366&jobListingId=1008406789899&jrtk=3-0-1goorp6m52f3a001-1goorp6mritnc800-0fa6a5942a4d0c54-,2.7,2.5,2.7,2.6,3.1,"Swindon, Regno Unito",Oltre 10.000 dipendenti,1792.0,Azienda quotata in borsa,Negozi di media e intrattenimento,Vendita al dettaglio e all''ingrosso,$ 100 - $ 500 milioni (USD)
6,6,High Society Catering Company,Other,"Londra, Inghilterra",NaN,Oltre 30 g,https://www.glassdoor.it/partner/jobListing.htm?pos=107&ao=1136043&s=58&guid=0000018631bc9a96b5de81e85a66ef30&src=GD_JOB_AD&t=SR&vt=w&ea=1&cs=1_50b80cfa&cb=1675871690366&jobListingId=1008218171277&jrtk=3-0-1goorp6m52f3a001-1goorp6mritnc800-b5f83bca420e7f1f-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,Hollister Co. Stores,Brand Specialist,"Londra, Inghilterra","<span class=""css-2lqh28 e1cjmv6j1"">3.5<i class=""css-0 e1cjmv6j0""></i></span>",4 g,https://www.glassdoor.it/partner/

## Istruzioni operative
E' necessario dichiare il data_asset_name e la expectation_suite_name che si vuole utilizzare

In [6]:
# inserisci qua il nome del file senza .csv
data_asset_name = 'scraping_all_londra_080223'

In [7]:
expectation_suite_name = 'Main'

In [8]:
context = gx.get_context()

2023-02-10T15:08:36+0000 - INFO - FileDataContext loading zep config
2023-02-10T15:08:36+0000 - INFO - GxConfig.parse_yaml() failed with errors - [{'loc': ('xdatasources',), 'msg': 'field required', 'type': 'value_error.missing'}]
2023-02-10T15:08:36+0000 - INFO - GxConfig.parse_yaml() returning empty `xdatasources`
2023-02-10T15:08:36+0000 - INFO - Loading 'datasources' ->
{}
2023-02-10T15:08:36+0000 - INFO - Loaded 'datasources' ->
{}
2023-02-10T15:08:37+0000 - INFO - Usage statistics is disabled; skipping initialization.


## Configurazione della datasource 

In [9]:
datasource_config: dict = {
    "name": "glassdoor_scraping",
    "class_name": "Datasource",
    "module_name": "great_expectations.datasource",
    "execution_engine": {
        "class_name": "PandasExecutionEngine",
        "module_name": "great_expectations.execution_engine",
    },
     "data_connectors": {
        "all": {
            "class_name": "InferredAssetFilesystemDataConnector",
            "base_directory": "../data",
            "default_regex": {"pattern": "(.*)\\.csv",
            "group_names": ["data_asset_name"]},
            #"batch_spec_passthrough": {
            #    "reader_method": "read_csv",
            #    "reader_options": {
            #        "header": True,
            #        "inferSchema": True,
            #    },
            #},
        }
    },
}

In [10]:
#check configurazione
context.test_yaml_config(yaml.dump(datasource_config))

Attempting to instantiate class from config...
	Instantiating as a Datasource, since class_name is Datasource
	Successfully instantiated Datasource


ExecutionEngine class name: PandasExecutionEngine
Data Connectors:
	all : InferredAssetFilesystemDataConnector

	Available data_asset_names (3 of 6):
		scraping_all_amsterdam_company_template (1 of 1): ['scraping_all_amsterdam_company_template.csv']
		scraping_all_amsterdam_job_template (1 of 1): ['scraping_all_amsterdam_job_template.csv']
		scraping_all_amsterdam_template (1 of 1): ['scraping_all_amsterdam_template.csv']

	Unmatched data_references (0 of 0):[]



In [11]:
## try to add
try:
    context.get_datasource(datasource_config["name"])
except ValueError:
    context.add_datasource(**datasource_config)
else:
    print(
        f"The datasource {datasource_config['name']} already exists in your Data Context!"
    )

The datasource glassdoor_scraping already exists in your Data Context!


In [12]:
# context.add_datasource(**datasource_config) # usa questo comando per sovrascrive la data source 

In [13]:
context.list_datasources()

[{'module_name': 'great_expectations.datasource',
  'name': 'glassdoor_scraping',
  'execution_engine': {'class_name': 'PandasExecutionEngine',
   'module_name': 'great_expectations.execution_engine'},
  'data_connectors': {'all': {'module_name': 'great_expectations.datasource.data_connector',
    'default_regex': {'group_names': ['data_asset_name'],
     'pattern': '(.*)\\.csv'},
    'class_name': 'InferredAssetFilesystemDataConnector',
    'base_directory': '../data'}},
  'class_name': 'Datasource'}]

## Creazione della Suite
Dopo aver configurato la data source creaimo le suite e le aspettive
[Galleria delle expectations](https://greatexpectations.io/expectations/)

In [14]:
context = gx.get_context()


# Feel free to change the name of your suite here. Renaming this will not remove the other one.
#expectation_suite_name = "Main"
try:
    suite = context.get_expectation_suite(expectation_suite_name=expectation_suite_name)
    print(f'Loaded ExpectationSuite "{suite.expectation_suite_name}" containing {len(suite.expectations)} expectations.')
except DataContextError:
    suite = context.create_expectation_suite(expectation_suite_name=expectation_suite_name)
    print(f'Created ExpectationSuite "{suite.expectation_suite_name}".')

2023-02-10T15:08:37+0000 - INFO - FileDataContext loading zep config
2023-02-10T15:08:37+0000 - INFO - GxConfig.parse_yaml() failed with errors - [{'loc': ('xdatasources',), 'msg': 'field required', 'type': 'value_error.missing'}]
2023-02-10T15:08:37+0000 - INFO - GxConfig.parse_yaml() returning empty `xdatasources`
2023-02-10T15:08:37+0000 - INFO - Loading 'datasources' ->
{}
2023-02-10T15:08:37+0000 - INFO - Loaded 'datasources' ->
{}
2023-02-10T15:08:37+0000 - INFO - Usage statistics is disabled; skipping initialization.
Created ExpectationSuite "Main".


## 1 Table Expectation

#### Questa Expectation verifica che le colonne nella tabella abbiano questo ordine
- company
- job_title
- location
- company_rating
- job_age
- job_link
- Sedeentrale'
- Dimensioni
- Fondatael'
- Tipo
- Entrate
- Opportunitài carriera'
- Stipendio benefit'
- Cultura valori'
- Dirigentienior'
- Equilibrioavoro/vita privata'
- Settore
- Segmento

In [15]:
# Create an Expectation
expectation_configuration = ExpectationConfiguration(
   # Name of expectation type being added
   expectation_type="expect_table_columns_to_match_ordered_list",
   # These are the arguments of the expectation
   # The keys allowed in the dictionary are Parameters and
   # Keyword Arguments of this Expectation Type
   kwargs={
      "column_list": [
                    'company',
                    'job_title',
                    'location',
                    'company_rating',
                    'job_age',
                    'job_link',
                    'Sede centrale',
                    'Dimensioni',
                    'Fondata nel',
                    'Tipo',
                    'Entrate',
                    'Opportunità di carriera',
                    'Stipendio e benefit',
                    'Cultura e valori',
                    'Dirigenti senior',
                    'Equilibrio lavoro/vita privata',
                    'Settore',
                    'Segmento']
   },
   # This is how you can optionally add a comment about this expectation.
   # It will be rendered in Data Docs.
   # See this guide for details:
   # `How to add comments to Expectations and display them in Data Docs`.
   meta={
      "notes": {
         "format": "markdown",
         "content": ""
      }
   }
)
# Add the Expectation to the suite
suite.add_expectation(expectation_configuration=expectation_configuration)

{"meta": {"notes": {"format": "markdown", "content": ""}}, "kwargs": {"column_list": ["company", "job_title", "location", "company_rating", "job_age", "job_link", "Sede centrale", "Dimensioni", "Fondata nel", "Tipo", "Entrate", "Opportunit\u00e0 di carriera", "Stipendio e benefit", "Cultura e valori", "Dirigenti senior", "Equilibrio lavoro/vita privata", "Settore", "Segmento"]}, "expectation_type": "expect_table_columns_to_match_ordered_list"}

## 2. Column Expectation

### Company

##### company  
    Questa Expectation verifica che la colonna sia di tipo stringa

In [16]:
expectation_configuration = ExpectationConfiguration(
   expectation_type="expect_column_values_to_be_in_type_list",
   kwargs={
        "column": "company",
      "type_list": ['str'],
   },
   # Note optional comments omitted
)
suite.add_expectation(expectation_configuration=expectation_configuration)

{"meta": {}, "kwargs": {"column": "company", "type_list": ["str"]}, "expectation_type": "expect_column_values_to_be_in_type_list"}

##### company  
    Questa Expectation verifica che la colonna non sia MAI nulla

In [17]:
expectation_configuration = ExpectationConfiguration(
   expectation_type="expect_column_values_to_not_be_null",
   kwargs={
      "column": "company",
   },
   # Note optional comments omitted
)
suite.add_expectation(expectation_configuration=expectation_configuration)

{"meta": {}, "kwargs": {"column": "company"}, "expectation_type": "expect_column_values_to_not_be_null"}

##### company  
       nuova expectation

### Job Title

##### job_title  
    Questa Expectation verifica che la colonna si di tipo stringa

In [18]:
expectation_configuration = ExpectationConfiguration(
   expectation_type="expect_column_values_to_be_in_type_list",
   kwargs={
        "column": "job_title",
      "type_list": ['str'],
   },
   # Note optional comments omitted
)
suite.add_expectation(expectation_configuration=expectation_configuration)

{"meta": {}, "kwargs": {"column": "job_title", "type_list": ["str"]}, "expectation_type": "expect_column_values_to_be_in_type_list"}

##### job_title  
    Questa Expectation verifica che la colonna non sia MAI nulla

In [19]:
expectation_configuration = ExpectationConfiguration(
   expectation_type="expect_column_values_to_not_be_null",
   kwargs={
      "column": "job_title",
   },
   # Note optional comments omitted
)
suite.add_expectation(expectation_configuration=expectation_configuration)

{"meta": {}, "kwargs": {"column": "job_title"}, "expectation_type": "expect_column_values_to_not_be_null"}

##### job_title  
    Questa Expectation verifica che la colonna non sia MAI nulla

### Salvataggio delle suite e creazione della documentazione
rimuovere il commento dalla prima riga per la costruzione della suite

In [20]:
# print(context.get_expectation_suite(expectation_suite_name=expectation_suite_name))
context.save_expectation_suite(expectation_suite=suite, expectation_suite_name=expectation_suite_name)

suite_identifier = ExpectationSuiteIdentifier(expectation_suite_name=expectation_suite_name)
context.build_data_docs(resource_identifiers=[suite_identifier])
context.open_data_docs(resource_identifier=suite_identifier)

### Apri questo link per vedere le expectations create

In [21]:
print('http://localhost:9000/view/great_expectations/uncommitted/data_docs/local_site/expectations/'+ expectation_suite_name +'.html')

http://localhost:9000/view/great_expectations/uncommitted/data_docs/local_site/expectations/Main.html


Se non si sta utilizzando il notebook dal docker si può arrivare al sito tramite il percorso 'great_expectations\uncommitted\data_docs\local_site'

## Creazione di un checkpoint

In [22]:
## checkpoint
yaml = YAML()

In [23]:
my_checkpoint_name = "main_checkpoint" # This was populated from your CLI command.

yaml_config = f"""
name: {my_checkpoint_name}
config_version: 1.0
class_name: SimpleCheckpoint
run_name_template: "%Y%m%d-%H%M%S-{data_asset_name}"
validations:
  - batch_request:
      datasource_name: glassdoor_scraping 
      data_connector_name: all
      data_asset_name: {data_asset_name}
      data_connector_query:
        index: -1
    expectation_suite_name: Main
"""

In [24]:
# Run this cell to print out the names of your Datasources, Data Connectors and Data Assets
pprint(context.get_available_data_asset_names())

{'glassdoor_scraping': {'all': ['scraping_all_londra_080223',
                                'scraping_all_londra_job_080223',
                                'scraping_all_londra_company_080223',
                                'scraping_all_amsterdam_template',
                                'scraping_all_amsterdam_job_template',
                                'scraping_all_amsterdam_company_template']}}


In [25]:
context.list_expectation_suite_names()

['Main']

In [26]:
main_checkpoint = context.test_yaml_config(yaml_config=yaml_config)

Attempting to instantiate class from config...
	Instantiating as a SimpleCheckpoint, since class_name is SimpleCheckpoint
	Successfully instantiated SimpleCheckpoint


Checkpoint class name: SimpleCheckpoint


In [27]:
#print(main_checkpoint.get_config(mode="yaml"))

In [28]:
# aggiungere un checkpoint
# se il checkpoint è già presente verrà sovrascritto
context.add_checkpoint(**yaml.load(yaml_config))
context.list_checkpoints()

['main_checkpoint']

[Come aggiungere dati o suite di convalida a un Checkpoint](https://docs.greatexpectations.io/docs/guides/validation/checkpoints/how_to_add_validations_data_or_suites_to_a_checkpoint

In [29]:
try:
    results = context.run_checkpoint(
        checkpoint_name="main_checkpoint",
        expectation_suite_name = 'Main'
    )
except:
    print('ko')

2023-02-10T15:08:38+0000 - INFO - 	5 expectation(s) included in expectation_suite.


Calculating Metrics:   0%|          | 0/23 [00:00<?, ?it/s]

In [30]:
# A questo punto la validazione è completata, puoi consultare i risultati

In [31]:
a = re.findall('[0-9]{8}-[0-9]{6}-' + data_asset_name + '/[0-9]{8}T[0-9]{6}.[0-9]{6}Z/[\S]{32}', str(list(results['run_results'])[0]))[0]
b = expectation_suite_name + '/' + a
path = r"../great_expectations/uncommitted/validations/" + b + '.json'

### Apri questo link per vedere le expectations create

In [32]:
print('http://localhost:9000/view/great_expectations/uncommitted/data_docs/local_site/expectations/'+ expectation_suite_name +'.html')

http://localhost:9000/view/great_expectations/uncommitted/data_docs/local_site/expectations/Main.html


### Se non stai usando docker vai in questo path 

In [33]:
print("great_expectations/uncommitted/data_docs/local_site/expectations/"+ expectation_suite_name +'.html')

great_expectations/uncommitted/data_docs/local_site/expectations/Main.html


### Apri questo link per vedere i risultati


In [34]:
print( "http://localhost:9000/view/great_expectations/uncommitted/data_docs/local_site/validations/" + b + '.html')

http://localhost:9000/view/great_expectations/uncommitted/data_docs/local_site/validations/Main/20230210-150838-scraping_all_londra_080223/20230210T150838.429284Z/97d7545684674b50fe8cbe7c68cd9ad0.html


### Se non stai usando docker vai in questo path 

In [35]:
print("great_expectations/uncommitted/data_docs/local_site/validations/" + b + '.html')

great_expectations/uncommitted/data_docs/local_site/validations/Main/20230210-150838-scraping_all_londra_080223/20230210T150838.429284Z/97d7545684674b50fe8cbe7c68cd9ad0.html
